# Get list of papers citing each top n most sited paper

AI research papers were previously collected to a file called "WoS_All_Most_cited.xls".
This notebook contains the code used to fech and connect citation info to papers in this data set.
This is done to N-most cited papers from the data set, because of the limited nuber of searches available.
SerpAPI is used to get this infromation from Google Scholar.
Pairs of papers are then created and written into files for use in a machine learning task.
One file contains pairs of papers between which exist a citation relationship, the other contains
randomly sampled pairs without a citation betweent them.

In [1]:
#!pip install xlwt

In [2]:
# Imports
import pandas as pd
import json

In [4]:
excel_filename = "WoS_All_Most_cited"
df = pd.read_excel(excel_filename + ".xls")
print(df.columns)
print(df.shape)

Index(['Unnamed: 0', 'Publication Type', 'Authors', 'Book Authors',
       'Book Editors', 'Book Group Authors', 'Author Full Names',
       'Book Author Full Names', 'Group Authors', 'Article Title',
       'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language',
       'Document Type', 'Conference Title', 'Conference Date',
       'Conference Location', 'Conference Sponsor', 'Conference Host',
       'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses',
       'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', '

### DOI is used for the search - Drop rows with NaN DOI and duplicates

In [5]:
nan_count = df['DOI'].isna().sum()
print(nan_count)

47


In [6]:
df = df[df['DOI'].notna()]
nan_count = df['DOI'].isna().sum()
print(nan_count)

0


In [7]:
print(df.shape)
df.drop_duplicates(subset=["DOI"], inplace=True)
print(df.shape)

(8793, 73)
(8791, 73)


In [8]:
df.iloc[0].DOI

'10.1109/TITS.2020.3024655'

In [9]:
nan_count = df['Abstract'].isna().sum()
print(nan_count)
df = df[df['Abstract'].notna()]
nan_count = df['Abstract'].isna().sum()
print(nan_count)

31
0


In [10]:
df.iloc[0].loc["Article Title"]

'Survey of Deep Reinforcement Learning for Motion Planning of Autonomous Vehicles'

In [11]:
df.sort_values('Times Cited, All Databases', axis=0, ascending=False, inplace=True)

In [12]:
df.head()['Times Cited, All Databases']

566     21220
4897    15358
270     10433
32       8598
3224     8036
Name: Times Cited, All Databases, dtype: int64

In [13]:
df.to_excel("WoS_All_Most_cited_nCitations_decending.xls")

C:\Users\Konna\AppData\Local\Temp\ipykernel_17988\3962508928.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("WoS_All_Most_cited_nCitations_decending.xls")


In [14]:
# Install SerpApi
!pip install google-search-results

In [15]:
# For reattempting failed searches
# If searcheas have already been done, and the files created during this exist, remove the quotations to activate this cell
# to not repead searches that have already been successful
already_searched_dois = []
"""
with open ("SearchIDs.txt", 'r') as s_id_file:
    while True:
        s_id = s_id_file.readline().rstrip("\n")
        if s_id != "":
            already_searched_dois.append(s_id.split(" ")[1])
        else:
            break
print(len(already_searched_dois))
print(already_searched_dois[0])
"""

2531
10.1038/nature14539


In [16]:
import os
#Add top cited papers
n_searches = 2550
import json
from serpapi import GoogleSearch
json_obj_list_search = []
with open("TopCited2.json", 'w') as json_file:
    with open ("SearchIDs2.txt", 'w') as s_id_file:
        json_file.write("{\n")
        n_discarded = 0
        for i in range(n_searches):
            doi = df.iloc[i]['DOI']
            params = {
                "engine": "google_scholar",
                "q": doi,
                "api_key": "9e76cb77baffe66b33207f987b436521cf9714b849e7c8b4abd59c4ff99c5308"
            }
            s_id = str("search_" + str(i) + ": " + doi)
            if doi not in already_searched_dois:
                try:
                    result = GoogleSearch(params).get_dict()
                    if type(result) is dict and result.get("organic_results") != None:
                        organic_result = result["organic_results"]
                        if len(organic_result) > 0 and result.get("search_metadata").get("status") == "Success" and organic_result[0].get("inline_links").get("cited_by") != None:
                            json_file.write(str('"' + s_id + '": '))
                            s_id_file.write(s_id + "\n")
                            json.dump(organic_result, json_file, indent=4)
                            json_file.write(",\n")
                    else:
                        n_discarded += 1
                except:
                    n_discarded += 1
        # Get rid of ,\n after last entry
        json_file.seek(json_file.tell() - 3, os.SEEK_SET)
        json_file.truncate()
        json_file.write("\n}")

'\nimport os\n#Add top cited papers\nn_searches = 2550\nimport json\nfrom serpapi import GoogleSearch\njson_obj_list_search = []\nwith open("TopCited2.json", \'w\') as json_file:\n    with open ("SearchIDs2.txt", \'w\') as s_id_file:\n        json_file.write("{\n")\n        n_discarded = 0\n        for i in range(n_searches):\n            doi = df.iloc[i][\'DOI\']\n            params = {\n                "engine": "google_scholar",\n                "q": doi,\n                "api_key": "9e76cb77baffe66b33207f987b436521cf9714b849e7c8b4abd59c4ff99c5308"\n            }\n            s_id = str("search_" + str(i) + ": " + doi)\n            if doi not in already_searched_dois:\n                try:\n                    result = GoogleSearch(params).get_dict()\n                    if type(result) is dict and result.get("organic_results") != None:\n                        organic_result = result["organic_results"]\n                        if len(organic_result) > 0 and result.get("search_metad

In [17]:
#print(n_discarded)

In [18]:
with open('TopCited.json') as f:
    searches = json.load(f)

In [19]:
print(searches["search_0: 10.1038/nature14539"])

[{'position': 0, 'title': 'Scaling learning algorithms towards AI', 'result_id': 'sjFkRZfnofYJ', 'type': 'Pdf', 'link': 'https://pdfs.semanticscholar.org/f01e/080777b59d6978e412ded8995edabbaa62f0.pdf', 'snippet': 'Scaling Learning Algorithms Towards AI Page 1 Scaling Learning Algorithms Towards AI Authors: Yoshua Bengio, Yann LeCun Presenter: Marilyn Vazquez George Mason University February 10, 2017 Bengion and LeCun (GMU) NLDA Seminar February 10, 2017 1 / 25 Page 2 Outline 1 Curse of Dimensionality 2 Shallow Learning 3 Deep Learning 4 Results 5 Conclusion Bengion and LeCun (GMU) NLDA Seminar February 10, 2017 2 / 25 Page 3 Curse of Dimensionality Curse of Dimensionality The curse of dimensionality can be viewed either as …', 'publication_info': {'summary': 'Y Bengio, Y LeCun - Large-scale kernel machines, 2007 - pdfs.semanticscholar.org', 'authors': [{'name': 'Y Bengio', 'link': 'https://scholar.google.com/citations?user=kukA0LcAAAAJ&hl=en&oi=sra', 'serpapi_scholar_link': 'https://se

In [20]:
# Create DataFrame where each paper has a cites_id
cites_ids = []
cites_dois = []
with open ("SearchIDs.txt", 'r') as s_id_file:
    while True:
        s_id = s_id_file.readline().rstrip("\n")
        if s_id != "":
            cites_ids.append(searches[s_id][0]["inline_links"]["cited_by"]["cites_id"])
            cites_dois.append(s_id.split(" ")[1])
        else:
            break
df_top_n = df[df['DOI'].isin(cites_dois)]
df_top_n = df_top_n.assign(Cites_id=cites_ids)
print(df_top_n.head()["Cites_id"])
print(df_top_n.head()["DOI"])

566     17771740241470960050
4897    18433510229623055695
270     17609610056545403599
32      18446742974550982994
3224    16986546914786142867
Name: Cites_id, dtype: object
566              10.1038/nature14539
4897                 10.1145/3065386
270              10.1038/nature14236
32      10.1016/j.neunet.2014.09.003
3224      10.1109/TPAMI.2017.2699184
Name: DOI, dtype: object


In [21]:
# For reattempting failed searches
with open('TopCitedCitedBy.json') as f:
    searches = json.load(f)
    
already_got_dois = []
with open ("CitedBySearchIDs.txt", 'r') as s_id_file:
    while True:
        s_id = s_id_file.readline().rstrip("\n")
        if s_id != "":
            if type(searches.get(s_id)) is dict and "error" not in searches.get(s_id).keys():
                already_got_dois.append(s_id.split(" ")[1])
        else:
            break
print(len(already_got_dois))

2723


In [22]:
discarded2 = 0
import os
import json
from serpapi import GoogleSearch
# Search for papers that cite each of the top papers
#Add top cited papers
json_obj_list_search = []
with open("TopCitedCitedBy2.json", 'w') as json_file:
    with open ("CitedBySearchIDs2.txt", 'w') as s_id_file:
        json_file.write("{\n")
        for i in range(df_top_n.shape[0]):
            doi = df_top_n.iloc[i]['DOI']
            params = {
                "engine": "google_scholar",
                "cites": df_top_n.iloc[i]['Cites_id'],
                "api_key": "6449cb6519444a9134b335b095de6085f36f74b2e7ebca097d744e231012dec3"
            }
            s_id = str("papers_citing_search_:" + str(i) + "_paper: " + doi)
            if doi not in already_got_dois:
                try:
                    s_result = GoogleSearch(params)
                    json_file.write(str('"' + s_id + '": '))
                    s_id_file.write(s_id + "\n")
                    json.dump(s_result.get_dict(), json_file, indent=4)
                    if i < df_top_n.shape[0]:
                        json_file.write(",\n")
                except:
                    discarded2 += 1
        json_file.write("\n}")

'\ndiscarded2 = 0\nimport os\nimport json\nfrom serpapi import GoogleSearch\n# Search for papers that cite each of the top papers\n#Add top cited papers\njson_obj_list_search = []\nwith open("TopCitedCitedBy2.json", \'w\') as json_file:\n    with open ("CitedBySearchIDs2.txt", \'w\') as s_id_file:\n        json_file.write("{\n")\n        for i in range(df_top_n.shape[0]):\n            doi = df_top_n.iloc[i][\'DOI\']\n            params = {\n                "engine": "google_scholar",\n                "cites": df_top_n.iloc[i][\'Cites_id\'],\n                "api_key": "6449cb6519444a9134b335b095de6085f36f74b2e7ebca097d744e231012dec3"\n            }\n            s_id = str("papers_citing_search_:" + str(i) + "_paper: " + doi)\n            if doi not in already_got_dois:\n                try:\n                    s_result = GoogleSearch(params)\n                    json_file.write(str(\'"\' + s_id + \'": \'))\n                    s_id_file.write(s_id + "\n")\n                    json.dum

In [23]:
with open('TopCitedCitedBy.json') as f:
    searches = json.load(f)

In [24]:
# Create DataFrame where each paper has a cites_id
cites_dois = []
with open ("CitedBySearchIDs.txt", 'r') as s_id_file:
    while True:
        s_id = s_id_file.readline().rstrip("\n")
        if s_id != "":
            if type(searches.get(s_id)) is dict and "error" not in searches.get(s_id).keys():
                cites_dois.append(s_id.split(" ")[1])
        else:
            break
df_top_n = df_top_n[df_top_n['DOI'].isin(cites_dois)]
print(df_top_n.head()["Cites_id"])
print(df_top_n.shape)

566     17771740241470960050
4897    18433510229623055695
270     17609610056545403599
32      18446742974550982994
3224    16986546914786142867
Name: Cites_id, dtype: object
(2531, 74)


In [25]:
#print(searches["papers_citing_search_:45_paper: 10.1016/j.jcp.2018.10.045"])
#print(searches["papers_citing_search_:39_paper: 10.1038/s41587-019-0036-z"])

In [26]:
# Add cited by information for each paper in top cited
cites_ids = []
cited_by_titles = []
cited_by_authors = []
paper_ids = []
with open ("CitedBySearchIDs.txt", 'r') as s_id_file:
    s_id = s_id_file.readline().rstrip("\n")
    if type(searches.get(s_id)) is dict and "error" not in searches.get(s_id).keys():
        organic_results = searches[str(s_id)]["organic_results"]
        while s_id != "" and type(organic_results) is list:
            temp_ids = []
            temp_titles = []
            temp_authors = []
            if s_id not in paper_ids and type(searches.get(s_id)) is dict and "error" not in searches.get(s_id).keys():
                paper_ids.append(s_id)
                if len(organic_results) > 0:
                    for result in organic_results:
                        if type(result) is dict and type(result.get("inline_links")) is dict and type(result.get("inline_links").get("cited_by")) is dict:
                            temp_ids.append(result.get("inline_links").get("cited_by").get("cites_id"))
                        if type(result) is dict and type(result.get("title")) is str:
                            temp_titles.append(result.get("title"))
                        if type(result) is dict and type(result.get("publication_info")) is dict and type(result.get("publication_info").get("authors")) is list:
                            temp_authors.append(result.get("publication_info").get("authors"))
                    cites_ids.append(temp_ids)
                    cited_by_titles.append(temp_titles)
                    cited_by_authors.append(temp_authors)
            s_id = s_id_file.readline().rstrip("\n")
            if type(searches.get(s_id)) is dict and "error" not in searches.get(s_id).keys():
                organic_results = searches[str(s_id)]["organic_results"]
            
df_top_n = df_top_n.assign(Cited_by_cites_ids=cites_ids)
df_top_n = df_top_n.assign(Cited_by_titles=cited_by_titles)
df_top_n = df_top_n.assign(Cited_by_authors=cited_by_authors)
df_top_n.head()[["Article Title", "Cites_id", "Cited_by_cites_ids", "Cited_by_titles", "Cited_by_authors"]]

,Article Title,Cites_id,Cited_by_cites_ids,Cited_by_titles,Cited_by_authors
566,Deep learning,17771740241470960050,"[559463397382443088, 6340776138936674554, 1676...",[Representation learning: A review and new per...,"[[{'name': 'Y Bengio', 'link': 'https://schola..."
4897,ImageNet Classification with Deep Convolutiona...,18433510229623055695,"[15853904744099473538, 4107479906416396359, 12...",[A comprehensive survey of image-based food re...,"[[{'name': 'GA Tahir', 'link': 'https://schola..."
270,Human-level control through deep reinforcement...,17609610056545403599,"[15107175476013490880, 16771076966357002396, 1...","[Cortical excitation and chronic pain, The rol...","[[{'name': 'M Zhuo', 'link': 'https://scholar...."
32,Deep learning in neural networks: An overview,18446742974550982994,"[12864264545474230024, 2690735549741753526, 11...","[Detail review on chemical, physical and green...","[[{'name': 'A Nazir', 'link': 'https://scholar..."
3224,DeepLab: Semantic Image Segmentation with Deep...,16986546914786142867,"[14852114597417537063, 16441576239143306181, 8...",[Review on remote sensing methods for landslid...,"[[{'name': 'A Mohan', 'link': 'https://scholar..."


In [27]:
print("Cites ID: ")
print(df_top_n.iloc[0]["Cites_id"])
print("\nCited by cites IDs")
print(df_top_n.iloc[0]["Cited_by_cites_ids"])
print("\nCited by titles")
print(df_top_n.iloc[0]["Cited_by_titles"])
print("\nCited by authors")
print(df_top_n.iloc[0]["Cited_by_authors"])

Cites ID: 
17771740241470960050

Cited by cites IDs
['559463397382443088', '6340776138936674554', '16766804411681372720', '7447715766504981253', '5331804836605365413', '13548556499559547747', '7817802389471101066', '15455167514840141912', '15744443975855027399', '8870515469873563525']

Cited by titles
['Representation learning: A review and new perspectives', 'Reservoir computing approaches to recurrent neural network training', 'Deep learning', 'Efficient estimation of word representations in vector space', 'Learning deep architectures for AI', 'Stacked denoising autoencoders: Learning useful representations in a deep network with a local denoising criterion.', 'Extracting and composing robust features with denoising autoencoders', 'Greedy layer-wise training of deep networks', 'Wide residual networks', '3D convolutional neural networks for human action recognition']

Cited by authors
[[{'name': 'Y Bengio', 'link': 'https://scholar.google.com/citations?user=kukA0LcAAAAJ&hl=en&oi=sra',

# Create pairs of papers based on citation info

In [28]:
from random import randint
# Remove duplicates
df_top_n.drop_duplicates(subset=["Cites_id"], inplace=True)
# Create positive and negative pairs based on cites_id
# [paper cited, citing paper]
p_pairs = []
n_pairs = []
citing_pair_ids = []
print(df_top_n.shape[0])
for i in range(df_top_n.shape[0]):
    paper_id = df_top_n.iloc[i]["Cites_id"]
    citing_ids = df_top_n.iloc[i]["Cited_by_cites_ids"]
    citing_papers = df_top_n[df_top_n['Cites_id'].isin(citing_ids)]
    n_p_pairs = 0
    for index, row in citing_papers.iterrows():
        citing_pair_ids.append(row.Cites_id)
        p_pairs.append([paper_id, row.Cites_id])
        n_p_pairs += 1   
    # sample the same number of negative examples
    not_citing_papers = df_top_n[~df_top_n['Cites_id'].isin([paper_id, *citing_ids])]
    n_indexes = []
    for j in range(n_p_pairs):
        while True:
            random_index = randint(0, not_citing_papers.shape[0] - 1)
            if (random_index not in n_indexes) and (random_index != i):
                n_indexes.append(random_index)
                break; 
    for index, row in not_citing_papers.iloc[n_indexes].iterrows():
        n_pairs.append([paper_id, row.Cites_id])

print(len(p_pairs))
print(len(n_pairs))

2510
2007
2007


In [29]:
df_top_n.head()["Cited_by_cites_ids"]

566     [559463397382443088, 6340776138936674554, 1676...
4897    [15853904744099473538, 4107479906416396359, 12...
270     [15107175476013490880, 16771076966357002396, 1...
32      [12864264545474230024, 2690735549741753526, 11...
3224    [14852114597417537063, 16441576239143306181, 8...
Name: Cited_by_cites_ids, dtype: object

In [30]:
# Add abstracts to pairs
# [paper cited, citing paper, paper cited abstract, citing paper abstract]
for i, pair in enumerate(p_pairs):
    p_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[0], "Abstract"].values[0])
    p_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[1], "Abstract"].values[0])
    p_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[0], "Authors"].values[0])
    p_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[1], "Authors"].values[0])
for i, pair in enumerate(n_pairs):
    n_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[0], "Abstract"].values[0])
    n_pairs[i].append(df_top_n.loc[df_top_n.Cites_id == pair[1], "Abstract"].values[0])

In [31]:
print(p_pairs[0])

['17771740241470960050', '13548556499559547747', 'Deep learning allows computational models that are composed of multiple processing layers to learn representations of data with multiple levels of abstraction. These methods have dramatically improved the state-of-the-art in speech recognition, visual object recognition, object detection and many other domains such as drug discovery and genomics. Deep learning discovers intricate structure in large data sets by using the backpropagation algorithm to indicate how a machine should change its internal parameters that are used to compute the representation in each layer from the representation in the previous layer. Deep convolutional nets have brought about breakthroughs in processing images, video, speech and audio, whereas recurrent nets have shone light on sequential data such as text and speech.', 'Sensors in everyday devices, such as our phones, wearables, and computers, leave a stream of digital traces. Personal sensing refers to col

In [32]:
print(n_pairs[0])

['17771740241470960050', '17826593427668687115', 'Deep learning allows computational models that are composed of multiple processing layers to learn representations of data with multiple levels of abstraction. These methods have dramatically improved the state-of-the-art in speech recognition, visual object recognition, object detection and many other domains such as drug discovery and genomics. Deep learning discovers intricate structure in large data sets by using the backpropagation algorithm to indicate how a machine should change its internal parameters that are used to compute the representation in each layer from the representation in the previous layer. Deep convolutional nets have brought about breakthroughs in processing images, video, speech and audio, whereas recurrent nets have shone light on sequential data such as text and speech.', 'This study presents a novel procedure based on ensemble empirical mode decomposition (EEMD) and optimized support vector machine (SVM) for 

In [33]:
print(p_pairs[1][1])

7817802389471101066


In [34]:
citing_ids = []
for pair in p_pairs:
    citing_ids.append(pair[1])
print(len(set(citing_ids)))

691


## Write pairs to files for later

In [35]:
# Using \t as delimeter to make sure the same symbol is not found in abstracts
with open ("Cited_Pairs_Authors.csv", 'w') as pairs_file:
    pairs_file.write("paper cited\tciting paper\tpaper cited abstract\tciting paper abstract\tpaper cited authors\tciting paper authors\n")
    for pair in p_pairs:
        for i, o in enumerate(pair):
            pairs_file.write(str(o))
            if i < len(pair) - 1:
                pairs_file.write("\t")
        pairs_file.write("\n")

with open ("Not_Cited_Pairs.csv", 'w') as pairs_file:
    pairs_file.write("paper cited\tciting paper\tpaper cited abstract\tciting paper abstract\n")
    for pair in n_pairs:
        for i, o in enumerate(pair):
            pairs_file.write(str(o))
            if i < len(pair) - 1:
                pairs_file.write("\t")
        pairs_file.write("\n")

'\nwith open ("Not_Cited_Pairs.csv", \'w\') as pairs_file:\n    pairs_file.write("paper cited\tciting paper\tpaper cited abstract\tciting paper abstract\n")\n    for pair in n_pairs:\n        for i, o in enumerate(pair):\n            pairs_file.write(str(o))\n            if i < len(pair) - 1:\n                pairs_file.write("\t")\n        pairs_file.write("\n")\n'